### 参数的更新方法

* 目的都一样：根据梯度更新参数以最小化损失。差别在于 **如何累积/调节梯度信息**、以及是否对每维做不同步长调整。
* 从简单到复杂：SGD → Momentum → AdaGrad → Adam（Adam 综合了动量与自适应学习率思想）。
### 1.SGD（随机梯度下降, stochastic gradient descent）
**更新规则**（包含学习率 $ \eta $）：

```
θ ← θ - η * g
```

其中 $g=\nabla_{\theta} L(\theta)$ 是当前小批量的梯度。

**特点**

* 最简单、直观。
* 学习率全局一致，手工调参重要。
* 对噪声敏感，收敛抖动较大，但有时能获得更好泛化。

**常用超参**

* 学习率 ($ \eta $)：通常 0.1、0.01、0.001 这类范围（视任务而定）。
* 常配合学习率衰减调度（step、cosine、warmup 等）。

**适用场景**

* 基线/简单模型，或在配合良好学习率schedule和正则化时表现优秀。

### 2.Momentum（动量）

**思想**：对过去梯度做指数加权平均，给更新“惯性”，减缓抖动并加速收敛于小曲率方向。

**更新规则**：

```
v ← μ * v - η * g
θ ← θ + v
```

通常 μ（动量系数）= 0.9，$g=\nabla_{\theta} L(\theta)$ 是当前小批量的梯度。

**特点**

* 能加速收敛（沿鞍点/长狭谷方向更快），抑制噪声。
* 与 SGD 相比更稳定、常见且简单。

**常用超参**

* μ ≈ 0.9，学习率比SGD可以略大/相同。

**适用场景**

* 大多数深度学习训练的默认选择（尤其配合batchnorm/weight decay）。

### 3.AdaGrad（自适应学习率）

**思想**：为每个参数维度维护累积平方梯度 $r$，用 $1/\sqrt{r}$ 缩放步长。对稀疏特征友好（稀疏任务如 NLP、推荐系统早期很流行）。

**更新规则**：

```
r ← r + g ⊙ g   (逐元素平方累加)
θ ← θ - η * g / (sqrt(r) + ε)
```

ε 为小常数（如 1e-8）防止除零。

**特点**

* 每个维度自适应学习率：累积梯度大的维度步长自动变小。
* 缺点：累积的 r 单调增长，导致学习率逐渐变得非常小（训练后期可能停滞）。

**常用超参**

* η 通常较小（如 0.01、0.1 需调试）；ε≈1e-8

**适用场景**

* 稀疏特征或初期快速收敛，但长期训练通常被 RMSProp/Adam 取代。

### 4.Adam（Adaptive Moment Estimation）

**思想**：同时使用一阶动量（梯度的指数移动平均）和二阶矩估计（平方梯度的指数移动平均），并做偏差校正。**结合了 Momentum 与 AdaGrad 的优点**。

**更新规则**（常见形式）：
<div align = "center">

| 名称          | 公式                                              | 含义                  |
| ----------- | ----------------------------------------------- | ------------------- |
| 一阶矩 $ m_t $ | $ m_t = \beta_1 m_{t-1} + (1 - \beta_1) g_t $   | 梯度的指数加权平均（动量）       |
| 二阶矩 $ v_t $ | $ v_t = \beta_2 v_{t-1} + (1 - \beta_2) g_t^2 $ | 梯度平方的指数加权平均（自适应学习率） |

</div>

由于在初期迭代时 $ m_t , v_t $ 都偏向于 0，
所以论文中提出了偏差修正项：

$$
\hat{m_t} = \frac{m_t}{1 - \beta_1^t}, \quad
\hat{v_t} = \frac{v_t}{1 - \beta_2^t}
$$

更新公式为：

$$
\theta_t = \theta_{t-1} - \eta \frac{\hat{m_t}}{\sqrt{\hat{v_t}} + \epsilon}
$$
常用 β1=0.9, β2=0.999, ε=1e-8。$t$ 为迭代次数。

**特点**

* 收敛快，超参较鲁棒（对初学者友好）。
* 对稀疏/噪声问题表现好；训练早期通常比 SGD 快很多。
* 已知问题：在某些大规模训练与泛化任务上，Adam 的泛化性能有时不如 SGD（因此实际工程中常用 Adam 做预训练或快速探索，然后用 SGD 微调）。
* 有改进版本：AdamW（把 weight decay 与动量分离，常用），AMSGrad（修正 v 单调性问题）等。

**常用超参**

* η 默认 1e-3 常用；β1=0.9, β2=0.999, ε=1e-8
* 如果使用 AdamW，最好把 weight decay 单独设置（例如 1e-2~1e-4 视任务而定）。

**适用场景**

* 快速原型、Transformer、NLP、很多 CV 任务训练初期。大多数现代网络训练常用 Adam 或 AdamW，之后根据需要切换到 SGD。

### 总结
* 目前并不存在能在所有问题中都表现良好的方法。这4种方法各有各的特点，都有各自擅长解决的问题和不擅长解决的问题。
<div align = "center">

|    优化器   | 是否有动量 | 是否自适应学习率 | 是否有偏差修正 | 特点                 |
| :------: | :---: | :------: | :-----: | :----------------- |
|    SGD   |   ❌   |     ❌    |    ❌    | 简单但震荡大             |
| Momentum |   ✅   |     ❌    |    ❌    | 更平滑、加速收敛           |
|  AdaGrad |   ❌   |     ✅    |    ❌    | 自动调整学习率但衰减过快       |
|  RMSprop |   ❌   |     ✅    |    ❌    | 解决 AdaGrad 学习率衰减问题 |
| **Adam** |   ✅   |     ✅    |    ✅    | 结合二者优点，是最主流的选择     |

</div>


In [ ]:
import numpy as np

class SGD:
    """随机梯度下降法（Stochastic Gradient Descent）
    SGD 是最基础的优化算法。
    它通过在每次迭代中使用小批量样本（mini-batch）来估计梯度，
    然后沿梯度反方向更新参数，从而最小化损失函数。

    优点：实现简单，计算效率高。
    缺点：容易陷入局部最小值，且更新方向波动较大。
    """

    def __init__(self, lr=0.01):
        # lr: 学习率（learning rate），控制每次参数更新的步长
        self.lr = lr
        
    def update(self, params, grads):
        # params: 模型参数（字典）
        # grads: 每个参数对应的梯度（字典）
        for key in params.keys():
            # 沿梯度反方向更新参数
            params[key] -= self.lr * grads[key] 
            # 公式：w = w - lr * dw


class Momentum:
    """Momentum 动量法
    Momentum 在 SGD 的基础上加入“惯性”概念。
    类比物理中的动量（momentum），
    当前的更新不仅依赖当前梯度，还依赖之前的更新方向，
    从而在陡峭但方向一致的区域加速收敛，在震荡方向上抑制波动。

    公式：
        v = μ * v - lr * grad
        param += v
    其中 μ 是动量系数（一般取 0.9）
    """

    def __init__(self, lr=0.01, momentum=0.9):
        self.lr = lr
        self.momentum = momentum
        self.v = None  # v 表示“速度”，即参数更新的累计动量
        
    def update(self, params, grads):
        if self.v is None:
            # 初始化速度向量，与参数形状一致
            self.v = {}
            for key, val in params.items():                                
                self.v[key] = np.zeros_like(val)
                
        for key in params.keys():
            # 更新速度：保留一部分旧速度，加上当前梯度方向的分量
            self.v[key] = self.momentum*self.v[key] - self.lr*grads[key] 
            # 更新参数：沿速度方向前进
            params[key] += self.v[key]


class AdaGrad:
    """AdaGrad 自适应学习率算法

    关键思想：对每个参数单独调整学习率。
    如果某个参数的梯度经常很大，说明它已经更新较多，应该降低学习率；
    相反，如果某个参数的梯度较小，则保持或稍微提高学习率。

    公式：
        h += grad^2
        param -= lr * grad / (sqrt(h) + ε)
    其中 h 是累积平方梯度，ε 是防止除零的微小常数。
    """

    def __init__(self, lr=0.01):
        self.lr = lr
        self.h = None  # 存储每个参数的累积平方梯度
        
    def update(self, params, grads):
        if self.h is None:
            self.h = {}
            for key, val in params.items():
                self.h[key] = np.zeros_like(val)
            
        for key in params.keys():
            # 累积平方梯度
            self.h[key] += grads[key] * grads[key]
            # 参数更新时除以根号累积梯度，抑制学习率
            params[key] -= self.lr * grads[key] / (np.sqrt(self.h[key]) + 1e-7)

class RMSprop:
    """RMSprop 优化算法

    RMSprop 是 AdaGrad 的改进版。
    AdaGrad 的问题是 h 会一直累积，导致学习率衰减过快；
    RMSprop 采用指数加权平均的方法，只考虑“最近”的梯度信息。

    公式：
        h = decay_rate * h + (1 - decay_rate) * grad^2
        param -= lr * grad / (sqrt(h) + ε)
    其中 decay_rate 一般取 0.9~0.99。
    """

    def __init__(self, lr=0.01, decay_rate = 0.99):
        self.lr = lr
        self.decay_rate = decay_rate
        self.h = None
        
    def update(self, params, grads):
        if self.h is None:
            self.h = {}
            for key, val in params.items():
                self.h[key] = np.zeros_like(val)
            
        for key in params.keys():
            # 对梯度平方做滑动平均（防止过度累积）
            self.h[key] *= self.decay_rate
            self.h[key] += (1 - self.decay_rate) * grads[key] * grads[key]
            # 更新参数，除以平方根平均梯度，控制学习率大小
            params[key] -= self.lr * grads[key] / (np.sqrt(self.h[key]) + 1e-7)

class Adam:
    """Adam 优化算法
    -----------------------------------------
    论文链接: Kingma & Ba, 2014
    (https://arxiv.org/abs/1412.6980v8)

    Adam（Adaptive Moment Estimation）综合了：
    - Momentum 的“动量”思想（使用一阶动量）
    - RMSprop 的“自适应学习率”思想（使用二阶动量）

    通过对梯度的“均值”和“方差”进行估计，
    Adam 能在训练过程中动态调整学习率，使得模型收敛更平稳、更高效。

    它目前是深度学习中最常用的优化算法之一。
    -----------------------------------------
    """

    def __init__(self, lr=0.001, beta1=0.9, beta2=0.999):
        """
        参数解释：
        - lr：初始学习率（通常设为 0.001）
        - beta1：一阶矩估计的衰减率（一般取 0.9）
        - beta2：二阶矩估计的衰减率（一般取 0.999）
        """
        self.lr = lr
        self.beta1 = beta1  # 控制一阶动量（平均梯度）的衰减速度
        self.beta2 = beta2  # 控制二阶动量（梯度平方）的衰减速度
        self.iter = 0       # 记录迭代次数，用于偏差修正
        self.m = None       # 一阶动量（Momentum）
        self.v = None       # 二阶动量（RMSprop）
        
    def update(self, params, grads):
        """
        参数：
        - params: 模型参数（dict），例如 {"W1": array, "b1": array, ...}
        - grads: 每个参数对应的梯度（dict）
        """
        if self.m is None:
            # 初始化动量项 m 和 v，与参数形状一致
            self.m, self.v = {}, {}
            for key, val in params.items():
                self.m[key] = np.zeros_like(val)
                self.v[key] = np.zeros_like(val)
        
        # 更新迭代次数
        self.iter += 1

        # 进行偏差修正（bias correction）
        # 这是 Adam 的关键部分，因为在初始几步中 m 和 v 都偏向 0。
        # 修正后的学习率 lr_t 保证算法在早期阶段仍能有效更新。
        lr_t = self.lr * np.sqrt(1.0 - self.beta2**self.iter) / (1.0 - self.beta1**self.iter)         
        
        for key in params.keys():
            # --- Step 1: 更新一阶动量（类似 Momentum）---
            # self.m[key] = β₁ * m + (1 - β₁) * grad
            # 也可以写成增量形式（如下），更数值稳定：
            self.m[key] += (1 - self.beta1) * (grads[key] - self.m[key])

            # --- Step 2: 更新二阶动量（类似 RMSprop）---
            # self.v[key] = β₂ * v + (1 - β₂) * (grad²)
            self.v[key] += (1 - self.beta2) * (grads[key]**2 - self.v[key])

            # --- Step 3: 利用校正后的学习率更新参数 ---
            # 参数更新公式：
            #   θ ← θ - α * (m̂ / (√v̂ + ε))
            # 其中 m̂ 和 v̂ 是经过偏差修正的动量项
            params[key] -= lr_t * self.m[key] / (np.sqrt(self.v[key]) + 1e-7)

##  一、梯度消失（Vanishing Gradient）

### 1. 概念

> 在反向传播时，梯度在层层传递中不断变小，最终靠近输入层的梯度趋近于 0，导致这些层几乎不更新权重。

也就是说：前面的层“几乎学不到东西”。

### 2. 数学原因（以链式法则解释）
反向传播时：
$$
\frac{\partial L}{\partial W_l} = \frac{\partial L}{\partial y_L} \cdot \frac{\partial y_L}{\partial y_{L-1}} \cdot \cdots \cdot \frac{\partial y_{l+1}}{\partial y_l} \cdot \frac{\partial y_l}{\partial W_l}
$$
这些乘积中往往包含很多“导数项”，比如激活函数的导数。
如果激活函数的导数 < 1（例如 sigmoid 在大多数输入区域导数约 0.1 以下），那么乘上几十层后，梯度会被指数级缩小：
$$
0.9^{50} ≈ 0.0059
$$
→ 梯度几乎消失。
### 3. 常见诱因
<div align = "center">

| 原因                   | 说明                  |
| -------------------- | ------------------- |
| 激活函数饱和（sigmoid/tanh） | 输入太大或太小 → 导数接近 0    |
| 权重初始化不当              | 初始值过小或过大 → 激活函数迅速饱和 |
| 网络太深                 | 乘积项太多，梯度自然衰减        |
| 不当的归一化               | 数据分布不稳定，信号/梯度易爆炸或消失 |

</div>

### 4. 影响

* 前层学习停滞，无法捕捉底层特征；
* 训练速度极慢；
* 即使训练损失下降，模型性能仍差。

### 5. 解决方法
<div align = "center">

| 方法                                  | 原理                                   |
| ----------------------------------- | ------------------------------------ |
| **ReLU 系列激活函数**                     | 线性区梯度=1，不易消失（ReLU, LeakyReLU, GELU等） |
| **Batch Normalization / LayerNorm** | 稳定激活分布，使梯度保持合适尺度                     |
| **残差结构（ResNet）**                    | 直接为梯度提供“捷径”路径，不会被多层乘积压缩              |
| **合理初始化（He/Xavier）**                | 保证激活和梯度方差在层间稳定                       |
| **梯度裁剪（gradient clipping）**         | 较少爆炸梯度（梯度过大时）但也辅助稳定                  |

</div>

## 二、表现力受限（Representation Limitation）

### 1. 概念

> 当网络的结构或激活函数设计使它无法有效表示复杂的非线性映射时，就称为 **表现力受限**。各层的激活值没有适当的广度。

简单说：
网络“梯度没问题，也能学”，但“学不到复杂函数”，表现能力太弱。

### 2. 原因举例
<div align = "center">

| 原因                  | 表现形式                           |
| ------------------- | ------------------------------ |
| **层数太少 / 神经元太少**    | 网络容量不足，无法近似复杂函数                |
| **激活函数过于线性**        | 如果所有层都是线性函数，整个网络等价于一个线性映射      |
| **权重正则化过强**         | 限制了参数变化空间，模型变得太平滑              |
| **共享结构过多（如过强参数共享）** | 模型灵活性下降                        |
| **过早饱和的激活函数**       | 输出几乎恒定，信息无法有效编码（类似梯度消失但属于表达问题） |

</div>

### 3. 数学直观解释

一个只有线性层的网络：
$$
f(x) = W_3 (W_2 (W_1 x)) = (W_3 W_2 W_1)x
$$
→ 仍然是线性变换。
→ 无论堆多少层，都只能拟合线性函数 → **表现力受限**。

非线性激活（如 ReLU）引入了“折线”或“分段”特性，使网络能表达复杂函数。

### 4. 解决思路
<div align = "center">

| 方向            | 方法                          |
| ------------- | --------------------------- |
| **增加非线性**     | 使用非线性激活（ReLU、GELU、Swish 等）  |
| **加深/加宽网络**   | 提高网络容量（更多层或更多神经元）           |
| **多分支或注意力结构** | 改善表达灵活性（ResNet、Transformer） |
| **正则化平衡**     | 保持表达力与泛化能力间的平衡              |

</div>

## 三、两者关系与区别
<div align = "center">

| 比较维度 | 梯度消失                | 表现力受限                |
| ---- | ------------------- | -------------------- |
| 本质问题 | **优化问题**（训练梯度无法传递）  | **结构问题**（网络无法表示复杂函数） |
| 原因   | 激活函数饱和、链式乘积过小、初始化问题 | 层太少、激活线性、参数约束太强      |
| 结果   | 模型“学不动”             | 模型“学不出复杂模式”          |
| 解决方向 | 改进训练信号传播            | 改进模型结构设计             |

</div>

## Batch Normalization（BN）——背景与方法简介

### 产生背景（为什么要提出 BN）

* **深层网络训练困难**：随着层数增多，层与层之间输入分布会随着前面参数更新而不断变化，导致后面层要不停适应新的输入分布，训练变慢且不稳定。
* **依赖学习率/初始化/正则化**：没有归一化时，常需要非常小的学习率、精心设计的权重初始化和/或强烈的正则化才能稳定训练（不同的激活函数往往使用不同分布的权重初始值）。

### 方法（BN 在一维激活上的核心步骤）

对某一层、某一维激活（或通道）在一个 mini-batch 上做以下处理：

1. 计算 batch 均值与方差：
   $$
   \mu_B = \frac{1}{m}\sum_{i=1}^m x_i,\qquad
   \sigma_B^2 = \frac{1}{m}\sum_{i=1}^m (x_i - \mu_B)^2
   $$
2. 归一化（zero-mean, unit-variance）：
   $$
   \hat{x}_i = \frac{x_i - \mu_B}{\sqrt{\sigma_B^2 + \epsilon}}
   $$
   其中 $\epsilon$ 是小常数防止除零。对输入数据进行均值为0，方差为1的正规化。
3. 可学习的尺度与偏移（恢复表达能力）：
   $$
   y_i = \gamma \hat{x}_i + \beta
   $$
   这里 $\gamma$（缩放）和 $\beta$（平移）是该维度的可学习参数，初始化通常 $\gamma=1,\ \beta=0$。

* Batch Norm的思路是调整各层的激活值分布使其拥有适当的广度。为此要向神经网络中插入对数据分布进行正规化的层，即Batch Normalization层。
  <div align="center">   <img src="data_mk/mk-2025-11-04-22-39-26.png" width="75%" style="margin: 20px 0;" /> </div>

### BN 的效果与好处
<div align = "center">

| 优点         | 说明               |
| ---------- | ---------------- |
| ✅ 加快收敛     | 缩放数据分布，避免梯度爆炸/消失 |
| ✅ 提高泛化     | 有轻微的正则化效果        |
| ✅ 可用较大学习率  | 网络更稳定            |
| ✅ 减少初始化敏感性 | 权重初值不再关键         |

</div>

### BN层的反向传播
略



In [ ]:
class BatchNormalization:
    """
    批量归一化（Batch Normalization）
    -------------------------------------------
    论文: Ioffe & Szegedy, 2015
    链接: http://arxiv.org/abs/1502.03167

    【主要思想】
    深度神经网络训练时，每层输入的分布会不断变化（称为“内部协变量偏移”），
    这会导致训练速度慢、对学习率敏感。
    
    BatchNorm 的目标是让每一层的输入分布保持稳定：
      - 对每个 mini-batch 的激活进行标准化（均值为 0，方差为 1）
      - 然后再引入可学习参数 gamma（缩放）和 beta（平移），
        保持模型的表达能力。

    好处：
    - 收敛更快，训练更稳定；
    - 可使用更大的学习率；
    - 对初始值不敏感；
    - 有一定正则化效果。
    -------------------------------------------
    """

    def __init__(self, gamma, beta, momentum=0.9, running_mean=None, running_var=None):
        """
        参数说明：
        gamma, beta : 可学习参数，用于缩放和平移（类似线性变换 y = γx + β）
        momentum : 用于计算测试阶段均值/方差的滑动平均系数
        running_mean / running_var : 测试时使用的全局统计量（在训练时不断更新）
        """
        self.gamma = gamma
        self.beta = beta
        self.momentum = momentum
        self.input_shape = None  # 输入的原始形状（卷积层为4D，全连接层为2D）

        # 测试阶段使用的均值和方差（推理阶段使用）
        self.running_mean = running_mean
        self.running_var = running_var  
        
        # backward 时要用到的中间变量（用于链式求导）
        self.batch_size = None
        self.xc = None   # 输入减去均值后的结果
        self.std = None  # 标准差
        self.dgamma = None
        self.dbeta = None

    # ------------------------ 前向传播 ------------------------
    def forward(self, x, train_flg=True):
        """
        x: 输入数据（N, D）或（N, C, H, W）
        train_flg: 是否为训练模式（True）或推理模式（False）
        """
        self.input_shape = x.shape
        if x.ndim != 2:  # 如果是卷积层输入(4维)
            N, C, H, W = x.shape
            # 变为二维 (N, C*H*W)，方便统一处理
            x = x.reshape(N, -1)

        out = self.__forward(x, train_flg)
        
        # 恢复成原来的形状
        return out.reshape(*self.input_shape)
            
    def __forward(self, x, train_flg):
        if self.running_mean is None:
            # 初始化全局统计量
            N, D = x.shape
            self.running_mean = np.zeros(D)
            self.running_var = np.zeros(D)
                        
        if train_flg:
            # ---------- 1️⃣ 计算当前 batch 的均值与方差 ----------
            mu = x.mean(axis=0)  # 均值 μ
            xc = x - mu          # 去中心化
            var = np.mean(xc**2, axis=0)  # 方差 σ²
            std = np.sqrt(var + 1e-7)     # 加上一个小常数防止除零
            xn = xc / std                 # 标准化结果

            # ---------- 2️⃣ 保存中间变量（用于反向传播） ----------
            self.batch_size = x.shape[0]
            self.xc = xc
            self.xn = xn
            self.std = std

            # ---------- 3️⃣ 更新全局统计量（测试时使用） ----------
            self.running_mean = self.momentum * self.running_mean + (1-self.momentum) * mu
            self.running_var = self.momentum * self.running_var + (1-self.momentum) * var            
        else:
            # ---------- 测试阶段使用滑动平均的统计量 ----------
            xc = x - self.running_mean
            xn = xc / (np.sqrt(self.running_var + 1e-7))
            
        # ---------- 4️⃣ 缩放和平移（恢复网络表达能力） ----------
        out = self.gamma * xn + self.beta 
        return out

    # ------------------------ 反向传播 ------------------------
    def backward(self, dout):
        """
        dout: 来自上层的梯度
        """
        if dout.ndim != 2:
            N, C, H, W = dout.shape
            dout = dout.reshape(N, -1)

        dx = self.__backward(dout)

        # 恢复形状
        dx = dx.reshape(*self.input_shape)
        return dx

    def __backward(self, dout):
        """
        反向传播推导（简化版）：
        对 BatchNorm 的反向传播推导公式如下：
            dγ = Σ(dout * xn)
            dβ = Σ(dout)
            dx = (1/N)*(γ/σ) * [N*dout - Σ(dout) - xn*Σ(dout*xn)]
        """
        # ---------- 1️⃣ 求 gamma, beta 的梯度 ----------
        dbeta = dout.sum(axis=0)
        dgamma = np.sum(self.xn * dout, axis=0)

        # ---------- 2️⃣ 对标准化输入的梯度 ----------
        dxn = self.gamma * dout  # 传回去的梯度乘以缩放因子

        # ---------- 3️⃣ 对输入 x 的梯度求解 ----------
        dxc = dxn / self.std
        dstd = -np.sum((dxn * self.xc) / (self.std * self.std), axis=0)
        dvar = 0.5 * dstd / self.std
        dxc += (2.0 / self.batch_size) * self.xc * dvar
        dmu = np.sum(dxc, axis=0)
        dx = dxc - dmu / self.batch_size
        
        # ---------- 4️⃣ 保存参数梯度 ----------
        self.dgamma = dgamma
        self.dbeta = dbeta
        
        return dx



## 权重衰减（Weight Decay）

**权重衰减（Weight Decay）** 是一种常用的 **正则化（regularization）** 技术，用于防止神经网络过拟合。
它通过在损失函数中增加一个与权重平方和成正比的惩罚项，使得模型在训练时不倾向于让权重参数过大。
数学表达式为：

$$
L_{\text{total}} = L_{\text{original}} + \frac{\lambda}{2} \sum_{i} W_i^2
$$

其中：

* $ L_{\text{original}} $：模型原本的损失（例如交叉熵）；
* $ W_i $：第 $i$ 个权重；
* $ \lambda $：权重衰减系数，控制正则化强度。

**作用：**

* 防止权重过大，减少模型复杂度；
* 提高泛化能力；
* 与 L2 正则化等价（L2 regularization）。


## 举例说明

假设：

* 模型预测的损失（交叉熵） = 0.8
* 权重矩阵平方和之和 = 4.0
* 正则化系数 λ = 0.1

则：
$$
\text{loss} = 0.8 + 0.5 * 0.1 * 4.0 = 0.8 + 0.2 = 1.0
$$

这样，网络不仅关注预测误差，也会惩罚过大的权重。


In [ ]:
def loss(self, x, t):
    """
    计算损失函数（包括权重衰减项）

    参数
    ----------
    x : ndarray
        输入数据（特征）
    t : ndarray
        教师标签（真实值）

    返回
    -------
    float
        总损失值（预测损失 + 权重衰减损失）
    """
    
    # 1. 前向传播，得到预测输出
    y = self.predict(x)

    # 2. 初始化权重衰减项
    weight_decay = 0

    # 3. 遍历每一层的权重参数，计算权重衰减项
    for idx in range(1, self.hidden_layer_num + 2):
        W = self.params['W' + str(idx)]  # 取出每一层的权重矩阵
        # 对所有权重的平方求和，再乘以正则化系数
        weight_decay += 0.5 * self.weight_decay_lambda * np.sum(W ** 2)

    # 4. 返回最终损失 = 原损失（如交叉熵） + 权重衰减项
    return self.last_layer.forward(y, t) + weight_decay


### **1. Dropout 的背景与动机**

在深度学习中，神经网络通常有大量参数，容易出现 **过拟合**（overfitting）：

* 网络在训练集上表现很好，但在测试集上性能下降。
* 特别是在小数据集或模型很深很宽时，过拟合更明显。

**Dropout** 是 Hinton 等人在 2014 年提出的一种 **正则化技术**，目的是降低过拟合，提高网络的泛化能力。

### **2. Dropout 的基本思想**

Dropout 的核心思想是 **在训练时随机“丢弃”一部分神经元**，让它们在该训练步骤中 **暂时不参与计算和更新**。

* 每个神经元以概率 (p) 被 **保留**（通常 (p=0.5)）。
* 剩下的神经元被置为 0，相当于临时从网络中移除。
* 在每一次训练迭代中，都会生成一个 **不同的子网络**。

这样做有几个效果：

1. **减少神经元之间的依赖**（co-adaptation），促使每个神经元学到更鲁棒的特征。
2. 类似于训练了很多不同的子网络，然后在测试时做平均，提高泛化能力。

### **3. Dropout 的数学实现**

假设某一层的输入为 $\mathbf{x}$，权重为 $\mathbf{W}$，输出为 $\mathbf{y}$：

#### 训练阶段

1. 生成一个与 $\mathbf{x}$ 同维度的随机掩码 $\mathbf{r}$：
   $$
   r_i \sim \text{Bernoulli}(p)
   $$
2. 对输入进行 element-wise 相乘：
   $$
   \tilde{\mathbf{x}} = \mathbf{x} \odot \mathbf{r}
   $$
3. 然后计算输出：
   $$
   \mathbf{y} = f(\mathbf{W}\tilde{\mathbf{x}} + \mathbf{b})
   $$
<div align="center">   <img src="data_mk/mk-2025-11-04-23-01-33.png" width="75%" style="margin: 20px 0;" /> </div>
#### 测试阶段

* 测试时不丢弃神经元，而是对输出按保留概率 (p) 缩放：
  $$
  \mathbf{y}_{\text{test}} = p \cdot f(\mathbf{W}\mathbf{x} + \mathbf{b})
  $$

> 有些框架（如 TensorFlow、PyTorch）在训练时直接缩放 1/p，这样测试时就无需再乘 p，叫做 **inverted dropout**。

### **4. Dropout 的效果**

* **训练时**：网络的容量看起来更小，因为每次迭代随机丢弃神经元。
* **测试时**：利用所有神经元进行预测，相当于做了模型集成（ensemble），提高泛化能力。

### **5. Dropout 的使用建议**

* 对于 **隐藏层**，通常 (p=0.5)。
* 对于 **输入层**，一般 (p=0.8) 或 0.9（因为输入信息很重要）。
* 只在训练时启用 Dropout，测试时关闭或缩放。
* 对卷积层，Dropout 可以作用于 **通道维度**，而不是每个像素。

In [ ]:
class Dropout:
    """
    Dropout 层实现
    参考论文: http://arxiv.org/abs/1207.0580

    作用：
        在训练时随机屏蔽一部分神经元的输出，以减少过拟合。
    """
    
    def __init__(self, dropout_ratio=0.5):
        """
        初始化 Dropout 层

        参数
        ----------
        dropout_ratio : float
            神经元被屏蔽（丢弃）的比例，范围 [0, 1)
            例如 dropout_ratio=0.5 表示每次随机丢弃 50% 的神经元
        """
        self.dropout_ratio = dropout_ratio  # 丢弃率
        self.mask = None                    # 掩码矩阵（用于记录哪些神经元被保留）

    def forward(self, x, train_flg=True):
        """
        前向传播

        参数
        ----------
        x : ndarray
            输入数据
        train_flg : bool
            是否为训练模式
            True：训练阶段，执行随机丢弃
            False：测试阶段，仅进行缩放

        返回
        ----------
        out : ndarray
            经过 Dropout 处理后的输出
        """
        if train_flg:
            # 生成一个与输入形状相同的随机矩阵，值在 [0,1)
            # mask 中 True 表示“保留神经元”，False 表示“丢弃神经元”
            self.mask = np.random.rand(*x.shape) > self.dropout_ratio
            
            # 将 mask 应用于输入：被丢弃的神经元输出置 0
            return x * self.mask
        else:
            # 测试阶段不进行随机丢弃，而是将输出按保留率缩放
            # 确保期望输出与训练阶段一致
            return x * (1.0 - self.dropout_ratio)

    def backward(self, dout):
        """
        反向传播

        参数
        ----------
        dout : ndarray
            来自上一层的梯度

        返回
        ----------
        dx : ndarray
            反向传播到前一层的梯度
        """
        # 仅将保留下来的神经元对应的梯度传递回去
        return dout * self.mask


## 🧠 一、什么是超参数（Hyperparameters）

在深度学习中，**超参数** 是指那些在训练之前就要设置好的参数，它们**不会在训练过程中自动更新**，而是**由人或搜索算法来决定**的。

它们通常决定了：

* 网络的结构（深度、宽度、激活函数等）；
* 训练的方式（学习率、批量大小、优化器参数等）；
* 正则化策略（Dropout 比例、权重衰减等）。

🔹 **超参数 ≠ 模型参数**

| 类型   | 例子                  | 是否自动学习     |
| ---- | ------------------- | ---------- |
| 模型参数 | 权重 `W`、偏置 `b`       | ✅ 由反向传播学习  |
| 超参数  | 学习率、batch size、层数、λ | ❌ 需手动或自动搜索 |

## ⚙️ 二、常见的深度学习超参数

| 类别       | 超参数                                                               | 说明         |
| -------- | ----------------------------------------------------------------- | ---------- |
| **优化相关** | 学习率 (`learning_rate`)、动量 (`momentum`)、优化器类型（SGD / Adam / RMSProp） | 影响收敛速度与稳定性 |
| **结构相关** | 层数、每层神经元数、卷积核大小、步幅、激活函数                                           | 决定模型表达能力   |
| **正则化**  | Dropout 比例、L2 权重衰减系数 (`λ`)、BatchNorm 参数                           | 防止过拟合      |
| **训练策略** | 批大小 (`batch_size`)、训练轮数 (`epochs`)、学习率调度策略                        | 控制训练效率与性能  |
| **数据增强** | 旋转角度、翻转概率、噪声强度                                                    | 提高模型泛化能力   |

## 🔍 三、超参数优化的常见方法

### 1️⃣ **手动搜索（Manual Search）**

最基础的方式，通过经验和直觉调参。
例如：

> “先用学习率 0.01 试试，不行就改 0.001。”

优点：简单直观；
缺点：耗时、主观、不系统。

### 2️⃣ **网格搜索（Grid Search）**

枚举所有参数组合，训练模型并比较性能。

🧮 示例：

```python
learning_rates = [0.1, 0.01, 0.001]
batch_sizes = [32, 64, 128]
```

→ 共 (3 \times 3 = 9) 组组合。

✅ 优点：全面搜索；
❌ 缺点：计算代价极高，尤其在多维空间中。

### 3️⃣ **随机搜索（Random Search）**

不是穷举，而是从参数空间随机采样若干组合。

📘 论文：《Random Search for Hyper-Parameter Optimization》（Bergstra & Bengio, 2012）

✅ 优点：

* 效率更高；
* 在相同预算下往往找到更优结果；
  ❌ 缺点：
* 仍需要较多实验。

### 4️⃣ **贝叶斯优化（Bayesian Optimization）**

使用概率模型（如高斯过程）建模“超参数 → 性能”的关系。
每次选择新的超参数组合时，根据历史结果预测“最可能更优”的区域。

📈 代表工具：

* **Spearmint**
* **Hyperopt**
* **Optuna**
* **Scikit-Optimize (skopt)**

✅ 优点：

* 收敛快，实验次数少；
  ❌ 缺点：
* 实现复杂，计算开销较大。

## 四、常用自动化工具库

| 工具                         | 特点                   |
| -------------------------- | -------------------- |
| **Optuna**                 | 高效、易用、支持分布式优化（🔥 推荐） |
| **Ray Tune**               | 适合大规模分布式实验           |
| **Hyperopt**               | 支持随机搜索与贝叶斯优化         |
| **Keras Tuner**            | TensorFlow 官方调参工具    |
| **Weights & Biases Sweep** | 云端可视化调参              |